## 두 번째 모델링 : 근로기간 Unknown 값을 knn imputer로 대체한 모델링

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('train_new_imputed.csv')

In [2]:
train.head()

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,12480000.0,3.0,6.0,2.0,72000000.0,18.90,15.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,14400000.0,5.0,10.0,0.0,130800000.0,22.33,21.0,10.0,0.0,373572.0,234060.0,0.0,0.0,1.0
2,12000000.0,3.0,5.0,0.0,96000000.0,8.60,14.0,1.0,0.0,928644.0,151944.0,0.0,0.0,0.0
3,14400000.0,3.0,8.0,0.0,132000000.0,15.09,15.0,1.0,0.0,325824.0,153108.0,0.0,0.0,2.0
4,18000000.0,5.0,7.0,2.0,71736000.0,25.39,19.0,8.0,0.0,228540.0,148956.0,0.0,0.0,1.0


In [3]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_ss = ss.fit_transform(X)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_ss, y, test_size=0.3, stratify=y, random_state=42)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score

In [6]:
def grid_search(model, params, random=False):
    clf = model
    if not random:
        grid = GridSearchCV(clf, params,
                                scoring='f1_macro', cv=5,
                                n_jobs=-1)
    else:
        grid = RandomizedSearchCV(clf, params, n_iter=10,
                                scoring='f1_macro', cv=5,
                                n_jobs=-1, random_state=42)
        
    grid.fit(X_train, y_train)
    
    
    best_model = grid.best_estimator_
    
    best_params = grid.best_params_
    print("최상의 매개변수:", best_params)
    
    best_score = grid.best_score_
    print("훈련 점수: {:.3f}".format(best_score))
    
    y_pred = best_model.predict(X_val)
    macro_f1_val = f1_score(y_val, y_pred, average='macro')
    print('테스트 세트 점수: {:.3f}'.format(macro_f1_val))

In [7]:
params = {'min_samples_leaf':[18,19,20,21,22],
          'min_impurity_decrease':[0.0],
          'max_features':['auto',0.6,0.61,0.62,0.63,0.64,0.65,0.66,0.67,0.68,0.69,0.70],
          'max_depth':[None,11,12,13,14,15,16,17,18],
          'class_weight' : [None, 'balanced']}

grid_search(DecisionTreeClassifier(random_state=42), params, random=True)

최상의 매개변수: {'min_samples_leaf': 19, 'min_impurity_decrease': 0.0, 'max_features': 0.68, 'max_depth': 17, 'class_weight': None}
훈련 점수: 0.668
테스트 세트 점수: 0.686


In [11]:
params = {'min_samples_leaf':[20],
          'min_impurity_decrease':[0],
          'max_features':[0.65,0.6,0.55],
          'max_depth':[85,90,95],
          'n_estimators' : [730,750,770]}

grid_search(RandomForestClassifier(n_jobs=-1, random_state=42), params, random=True)

최상의 매개변수: {'n_estimators': 770, 'min_samples_leaf': 20, 'min_impurity_decrease': 0, 'max_features': 0.65, 'max_depth': 90}
훈련 점수: 0.688
테스트 세트 점수: 0.708


최상의 매개변수: {'n_estimators': 700, 'min_samples_leaf': 20, 'min_impurity_decrease': 0.0, 'max_features': 0.5, 'max_depth': 80}
훈련 점수: 0.651
테스트 세트 점수: 0.661

36분 소요

최상의 매개변수: {'n_estimators': 750, 'min_samples_leaf': 20, 'min_impurity_decrease': 0, 'max_features': 0.6, 'max_depth': 90}
훈련 점수: 0.668
테스트 세트 점수: 0.683

45분.. 시간상 후행 작업 중단

최상의 매개변수: {'n_estimators': 770, 'min_samples_leaf': 20, 'min_impurity_decrease': 0, 'max_features': 0.65, 'max_depth': 90}
훈련 점수: 0.688
테스트 세트 점수: 0.708

In [12]:
params = {'n_estimators' : [130,150,170],
          'learning_rate' : [0.18,0.2,0.22],
          'max_depth' : [33,34,35,36,37],
          'objective' : ['multi:softmax']}

grid_search(XGBClassifier(random_state=42, n_jobs=-1), params, random=True)

최상의 매개변수: {'objective': 'multi:softmax', 'n_estimators': 130, 'max_depth': 34, 'learning_rate': 0.18}
훈련 점수: 0.794
테스트 세트 점수: 0.794


최상의 매개변수: {'objective': 'multi:softprob', 'n_estimators': 200, 'max_depth': 40, 'learning_rate': 0.2}
훈련 점수: 0.794
테스트 세트 점수: 0.792

10분 소요

최상의 매개변수: {'objective': 'multi:softmax', 'n_estimators': 150, 'max_depth': 35, 'learning_rate': 0.2}
훈련 점수: 0.795
테스트 세트 점수: 0.794

9분 -> 성능 변화 거의 없음. 이거 사용

최상의 매개변수: {'objective': 'multi:softmax', 'n_estimators': 130, 'max_depth': 34, 'learning_rate': 0.18}
훈련 점수: 0.794
테스트 세트 점수: 0.794

In [13]:
params = {'n_estimators' : [340,350,360],
          'learning_rate' : [0.05,0.06,0.07,0.08,0.09],
          'max_depth' : [8,9,10,11,12],
          'num_leaves' : [43,45,47]}

grid_search(LGBMClassifier(objective='multiclass', random_state=42, n_jobs=-1), params, random=True)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005377 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1456
[LightGBM] [Info] Number of data points in the train set: 67405, number of used features: 13
[LightGBM] [Info] Start training from score -1.747717
[LightGBM] [Info] Start training from score -1.206424
[LightGBM] [Info] Start training from score -1.248802
[LightGBM] [Info] Start training from score -1.975557
[LightGBM] [Info] Start training from score -2.572111
[LightGBM] [Info] Start training from score -3.897369
[LightGBM] [Info] Start training from score -5.434895
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
최상의 매개변수: {'num_leaves': 45, 'n_e

최상의 매개변수: {'num_leaves': 40, 'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.05}
훈련 점수: 0.779
테스트 세트 점수: 0.786

6분 소요

최상의 매개변수: {'num_leaves': 45, 'n_estimators': 350, 'max_depth': 10, 'learning_rate': 0.07}
훈련 점수: 0.785
테스트 세트 점수: 0.785

6분 -> 성능변화 거의 없음. 이거 사용

최상의 매개변수: {'num_leaves': 45, 'n_estimators': 360, 'max_depth': 12, 'learning_rate': 0.08}
훈련 점수: 0.785
테스트 세트 점수: 0.785

In [15]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

X_ss = pd.DataFrame(X_ss, columns=X.columns)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

f1_macro_scores = []

def skf_score(model):
    for train_idx, valid_idx in skf.split(X_ss, y):
        X_train = X_ss.iloc[train_idx]
        X_val = y.iloc[train_idx]

        y_train = X_ss.iloc[valid_idx]
        y_val = y.iloc[valid_idx]

        model.fit(X_train, X_val)

        pred = model.predict(y_train)

        f1_macro = f1_score(y_val, pred, average='macro')
        f1_macro_scores.append(f1_macro)
    
    average_f1_macro = np.mean(f1_macro_scores)

    print("Average F1-macro score:", average_f1_macro)
    try:
        if model.feature_importances_.any():
            feature_importances = model.feature_importances_
            print("\n",'-'*10,'특성중요도','-'*10)
            for feature, importance in zip(X_ss.columns, feature_importances):
                print(f"{feature}: {importance}")
    except:
        None

In [16]:
skf_score(DecisionTreeClassifier(min_samples_leaf=19, min_impurity_decrease=0, max_features=0.68,
                                 max_depth=17, class_weight=None, random_state=42))

Average F1-macro score: 0.6966618175326169

 ---------- 특성중요도 ----------
대출금액: 0.057607057926605525
대출기간: 0.0325371434609713
근로기간: 0.0016173114754242752
주택소유상태: 0.0010188254708840736
연간소득: 0.006211342461809011
부채_대비_소득_비율: 0.01409643635137708
총계좌수: 0.0027563047729122062
대출목적: 0.002383459725660737
최근_2년간_연체_횟수: 0.0004418932682137579
총상환원금: 0.5502363624581739
총상환이자: 0.33109386262796814
총연체금액: 0.0
연체계좌수: 0.0


In [17]:
skf_score(RandomForestClassifier(n_estimators=770, min_samples_leaf=20, min_impurity_decrease=0, 
                                 max_features=0.65, max_depth=90, random_state=42, n_jobs=-1))

Average F1-macro score: 0.7167816163115104

 ---------- 특성중요도 ----------
대출금액: 0.050826232153413406
대출기간: 0.04547071830990656
근로기간: 0.0028354894652486483
주택소유상태: 0.0016484482560358566
연간소득: 0.017670662409359016
부채_대비_소득_비율: 0.011231214685143194
총계좌수: 0.005353223319572358
대출목적: 0.004178031471702471
최근_2년간_연체_횟수: 0.001274893230546957
총상환원금: 0.4528059074110537
총상환이자: 0.4067051792880177
총연체금액: 0.0
연체계좌수: 0.0


In [18]:
skf_score(XGBClassifier(objective='multi:softmax', n_estimators=200, max_depth=40, 
          learning_rate=0.2, n_jobs=-1, random_state=42))

Average F1-macro score: 0.7466399349797003

 ---------- 특성중요도 ----------
대출금액: 0.029054811224341393
대출기간: 0.5254350900650024
근로기간: 0.01224927231669426
주택소유상태: 0.012388147413730621
연간소득: 0.018565339967608452
부채_대비_소득_비율: 0.01300264522433281
총계좌수: 0.013052946887910366
대출목적: 0.017211686819791794
최근_2년간_연체_횟수: 0.01752505637705326
총상환원금: 0.14957977831363678
총상환이자: 0.16137993335723877
총연체금액: 0.01628206856548786
연체계좌수: 0.014273281209170818


In [19]:
skf_score(LGBMClassifier(objective='multiclass', num_leaves=45, n_estimators=360, max_depth=12, 
               learning_rate=0.08, n_jobs=-1, random_state=42))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1462
[LightGBM] [Info] Number of data points in the train set: 86663, number of used features: 13
[LightGBM] [Info] Start training from score -1.747730
[LightGBM] [Info] Start training from score -1.206395
[LightGBM] [Info] Start training from score -1.248807
[LightGBM] [Info] Start training from score -1.975538
[LightGBM] [Info] Start training from score -2.572234
[LightGBM] [Info] Start training from score -3.897282
[LightGBM] [Info] Start training from score -5.434888
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003863 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1469
[LightGBM] [Info] Number of data points in the train set: 86663, number o

model|k-Fold|Sk-Fold
-|-|-
DecisionTree Classifier|0.686|0.6966618175326169
RandomForest Classifier|0.708|0.7167816163115104
XGBoost Classifier|0.794|0.770969787812605
Light GBM Classifier|0.785|0.7761664858844045 --> 최고 성능